<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Guided Problem Set 2: Error Propagation</h1>


<a name='section_2_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P2.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_2_1">P2.1 Error Propagation - A Simple Example</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_2_1">P2.1 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_2_2">P2.2 Error Propagation - A More Complicated Example</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_2_2">P2.2 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_2_3">P2.3 Johnson Noise</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_2_3">P2.3 Problems</a></td>
    </tr>
</table>

<h3>New Library</h3>

**Using Jupyter Notebook Locally**

Starting in this section of the course, you will need access to the `lmfit` library. If you are running Juypter locally, you don't need to do anything, as this library was installed during the initial setup of your 8.S50x environment. So, you can ignore the first code cell below and jump right to the import code.

If you didn't perform this installation (or others), then activate your 8.S50x conda environment and execute the following installations:

<pre>
conda install lmfit
</pre>


**Using Colab**

However, if you are running this notebook in a Colab environment, the procedure is slightly different. Unlike the libraries that were used previously, `lmfit` is not included in the default Colab environment. To do this installation, you must run the `!pip install lmfit` command in the code cell below.

In [ ]:
#>>>RUN: P2.0-runcell00

!pip3 install lmfit 

In [ ]:
#>>>RUN: P2.0-runcell01

import numpy as np                 #https://numpy.org/doc/stable/
import matplotlib.pyplot as plt    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import scipy.stats                 #https://docs.scipy.org/doc/scipy/reference/stats.html
from scipy.integrate import trapz  #https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.integrate.trapz.html
from lmfit.models import Model     #https://lmfit.github.io/lmfit-py/model.html

In [ ]:
#>>>RUN: P2.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title

<a name='section_2_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P2.1 Error Propagation - A Simple Example</h2>    

| [Top](#section_2_0) | [Previous Section](#section_2_0) | [Problems](#problems_2_1) | [Next Section](#section_2_2) |


In [ ]:
#>>>RUN: P2.1-runcell01

def f(x, y):
    return x + y

def delta_f(delta_x, delta_y):
    return np.sqrt((delta_x**2.)+(delta_y**2.))

x_val = 5.
x_err = 2.

y_val = 9.
y_err = 2.

print("f(x) = %f +/- %f" % (f(x_val, y_val), delta_f(x_err, y_err)))


In [ ]:
#>>>RUN: P2.1-runcell02

np.random.seed(2)
N_SAMPLES = 10000
N_BINS = 100
x_samples = np.random.normal(loc = x_val, scale = x_err, size = N_SAMPLES)
y_samples = np.random.normal(loc = y_val, scale = y_err, size = N_SAMPLES)

f_samples = f(x_samples, y_samples)

print("observed: f(x) = %f +/- %f" % (np.mean(f_samples), np.std(f_samples)))
print("expected: f(x) = %f +/- %f" % (f(x_val, y_val), delta_f(x_err, y_err)))

#MAKING A PLOT
counts, bin_edges = np.histogram(f_samples, bins = N_BINS, density = True)
bin_centers = 0.5*(bin_edges[:-1]+bin_edges[1:])

#Plotting the data
#Alternatively use: #plt.plot(bin_centers,counts)
plt.step(bin_edges[1:],counts)

#Plotting Gaussian with mean and std given by  f and delta_f
plt.plot(bin_centers, scipy.stats.norm.pdf(bin_centers, loc = f(x_val, y_val), scale = delta_f(x_err, y_err)))

<a name='problems_2_1'></a>     

| [Top](#section_2_0) | [Restart Section](#section_2_1) | [Next Section](#section_2_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 2.1.1</span>

Let $g(x_1,x_2,x_3...x_n) = x_1 + x_2 + x_3 + ... + x_n$ be a function of $n$ variables, where each variable has an error of 5. Complete the code below to manually compute the error of $g$ with 2 variables, 101 variables, and 5012 variables. 


Hint: Begin with the definition for error propagation. What does it reduce to when the `n` errors are the same?

$$\Delta g(x_1, x_2, x_3...x_n) = \sqrt{(\partial g / \partial x_1)^2(\Delta x_1)^2 + (\partial g / \partial x_2)^2(\Delta x_2)^2 +...+(\partial g / \partial x_1)^2(\Delta x_1)^2}$$

In [ ]:
#>>>PROBLEM: P2.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def delta_g(delta_x, n):
  ####################
  # Insert Code Here #
  ####################
  return 

x_err = 5.

n=2
print("g_err(n=2) = %f" % (delta_g(x_err, n)))
n=101
print("g_err(n=101) = %f" % (delta_g(x_err, n)))
n=5012
print("g_err(n=5012) = %f" % (delta_g(x_err, n)))


>#### Follow-up 2.1.1a (ungraded)
>   
>Why is it important to minimize the number of variables that contribute to your uncertainty? A lot of functions in physics analysis are dependent on not just a linear combination of terms, but perhaps an exponential one. What would the error of this look like with $n$ variables?


<a name='section_2_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P2.2 Error Propagation - A More Complicated Example</h2>    

| [Top](#section_2_0) | [Previous Section](#section_2_1) | [Problems](#problems_2_2) | [Next Section](#section_2_3) |


<a name='problems_2_2'></a>    

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 2.2.1</span>

Now let's take $h(x,y) = (\sqrt{|x|} + \sqrt{|y|})\cdot (x - y)$

Fill in the following code cell to compute the error on `h(x,y)` using the same values of $x$ and $y$ (and their respective errors) from the example code in P2.1. Take the stdev of `h(x,y)` and add it to the mean of `h(x,y)` to get an upper bound that is "one error away". What is this value?

Enter your answer as a number with precision 1e-1.

**NOTE:** You must specifically use the random seed and number of samples defined in the code below, for comparison with the answer checker.

In [ ]:
#>>>PROBLEM: P2.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

np.random.seed(2)
N_SAMPLES = 100000
N_BINS = 100
x_val = 5.
x_err = 2.
y_val = 9.
y_err = 2.
x_samples = np.random.normal(loc = x_val, scale = x_err, size = N_SAMPLES)
y_samples = np.random.normal(loc = y_val, scale = y_err, size = N_SAMPLES)

def h(x,y):
    return (np.sqrt(np.abs(x))+np.sqrt(np.abs(y)))*(x-y)


####################
# Insert Code Here #
####################

def h_val_err(ix, iy):
    h_samples = None # Placeholder Value - Fill in the correct line
    h_val = None # Placeholder Value - Fill in the correct line
    h_err = None # Placeholder Value - Fill in the correct line
    return h_val, h_err


####################


h_val, h_err = h_val_err(x_samples, y_samples)

print("observed: h(x) upper bound", h_val + h_err)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 2.2.2</span>

Write a function that computes the error in `h(x,y)`. What is the expected error, based on the propagation of error formula? Does this match the error that you obtained in the previous problem?


Use the values given previously to evaluate `delta_h(x_val,y_val,delta_x,delta_y)`:

<pre>
x_val = 5.
x_err = 2.
y_val = 9.
y_err = 2.
</pre>

Enter your answer as a number with precision 1e-2.

In [ ]:
#>>>PROBLEM: P2.2.2

#defining delta_h
####################

def delta_h(x_val,y_val,delta_x,delta_y):
    x_deriv = 0 #YOUR CODE HERE
    y_deriv = 0 #YOUR CODE HERE
    return np.sqrt(x_deriv**2. * delta_x**2. + y_deriv**2. * delta_y**2.)


print("observed: h(x) = %f +/- %f" % (h_val, h_err))
print("expected: h(x) = %f +/- %f" % (h(x_val, y_val), delta_h(x_val,y_val,x_err,y_err)))

>#### Follow-up 2.2.2a (ungraded)
>   
>Approximate the data as a Gaussian with mean and variance. How well does this Gaussian compare with the data? How does this Gaussian depend on the mean and variance of the underlying distributions?


<a name='section_2_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P2.3 Johnson Noise</h2>   

| [Top](#section_2_0) | [Previous Section](#section_2_2) | [Problems](#problems_2_3) |


In [ ]:
#>>>RUN: P2.3-runcell01

frequency = np.array([   200.,    300.,    400.,    500.,    600.,    700.,    800.,
          900.,   1000.,   1100.,   1200.,   1300.,   1400.,   1500.,
         1700.,   2000.,   3000.,   4000.,   5000.,   7000.,  10000.,
        13000.,  15000.,  17000.,  20000.,  25000.,  30000.,  35000.,
        40000.,  45000.,  50000.,  55000.,  60000.,  65000.,  70000.,
        75000.,  80000.,  85000.,  90000.,  95000., 100000.])

gain = np.array([  1.56572199,   7.56008454,  24.23507344,  58.36646477,
       119.11924863, 215.75587662, 354.79343025, 517.34083494,
       679.81395988, 805.18954729, 877.53623188, 944.14612835,
       951.12203586, 981.66551215, 976.08071562, 971.57565072,
       991.33195051, 974.54482165, 968.02100388, 970.96127868,
       972.70192708, 980.9122768 , 983.62597547, 981.85446382,
       964.75994752, 984.27991886, 959.44478862, 975.87335094,
       906.24841379, 831.8699187 , 695.5940221 , 562.69096627,
       426.50959034, 328.93671408, 248.14630158, 198.16023325,
       150.59357167, 121.00349255, 100.86777721,  79.42663031,
        63.20952534])

gain_uncertainty = np.array([5.21317443e-03, 3.11522352e-02, 1.17453781e-01, 1.54063502e-01,
       1.27335068e+00, 1.27124575e+00, 1.62862522e+00, 8.07632112e-01,
       1.39800408e+00, 1.52872753e+00, 9.26100943e-01, 2.07700290e+00,
       2.41624111e+00, 2.48737608e+00, 2.66446131e+00, 6.30956544e+00,
       2.48543922e+00, 5.85031911e+00, 5.36245736e+00, 5.03316166e+00,
       5.96042863e+00, 1.80119083e+00, 2.19189309e+00, 4.76416499e+00,
       2.60518705e+00, 8.91016625e-01, 8.68517783e-01, 7.60893395e-02,
       1.12595429e+00, 9.59211786e-01, 2.11207039e+00, 1.54206027e+00,
       6.15658573e-01, 2.21068956e+00, 1.93131996e+00, 1.17159272e+00,
       1.02084395e+00, 6.45939329e-01, 1.15822783e+00, 1.50426555e-01,
       2.64213908e-01])

resistance = np.array([477.1e3, 810e3, 99.7e3, 502.3e3, 10.03e3]) 
resistance_uncertainty = np.array([0.2e3, 2e3, 0.2e3, 0.3e3, 0.3e3])

capacitance = 125e-12
capacitance_uncertainty = 14e-12

#measured Johnson Noise and uncertainty
#note that this is the value of the left side of the equation shown above
v2rmsd4t = np.array([2.57337556e-08, 1.96214066e-08, 2.21758082e-08, 2.38320749e-08,
       7.31633110e-09])
v2rmsd4t_uncertainty = np.array([1.25267830e-09, 1.46644504e-09, 1.08426579e-09, 1.77538860e-09,
       2.07583938e-10])


In [ ]:
#>>>RUN: P2.3-runcell02

from scipy.integrate import trapz

def mc_compute(freq, gain, gain_error, r, rerr, cap, cap_err, n_samp):
    samples = []
    for k in range(n_samp):
        mc_gain = gain + np.random.normal(len(gain))*gain_error
        mc_r = r + rerr*np.random.normal(1)
        mc_cap = cap + cap_err*np.random.normal(1)
        mc_integrand = mc_gain**2.0/(1+ (2*np.pi*mc_r*mc_cap*freq)**2.0)
        mc_int = scipy.integrate.trapz(mc_integrand, freq)
        samples.append(mc_r*mc_int)
    return np.array(samples)

In [ ]:
#>>>RUN: P2.3-runcell03

rgr = []
rgr_unc = []
for k in range(5):
    samples = mc_compute(frequency, gain, gain_uncertainty, resistance[k], resistance_uncertainty[k], capacitance, capacitance_uncertainty,100)
    rgr.append(np.mean(samples))
    rgr_unc.append(np.std(samples))
rgr = np.array(rgr)  
rgr_unc = np.array(rgr_unc)

In [ ]:
#>>>RUN: P2.3-runcell04

plt.errorbar(rgr, v2rmsd4t, yerr = v2rmsd4t_uncertainty, xerr= rgr_unc, fmt = 'o' )
plt.show()

In [ ]:
#>>>RUN: P2.3-runcell05

from lmfit.models import Model

def linear_model(x, k, b):
    return x/k+b

lmod = Model(linear_model)
lmod.set_param_hint(name = 'k', value = 1)
lmod.set_param_hint(name = 'b', value = 1)
result = lmod.fit(1e-15*rgr, x = v2rmsd4t*1e8, weights = 1/(rgr_unc*1e-15))
print(result.fit_report())
result.plot_fit()
plt.show()

In [ ]:
#>>>RUN: P2.3-runcell06

m_earth = 5.9722 * 10**24 
m_earth_unc = 6*10^22

r_earth = 6.371 * 10**6
r_earth_unc = 100000

#length and unc
l = [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0]
l_unc = [.01, .02, .03, .03, .03, .04, .03, .04, .05, .04]

T = [0.6468, 0.9317, 1.1352, 1.4553, 1.6181, 1.782,  1.969,  2.068,  2.211,  2.255 ]
#Note, we are assuming that the experiment is run for a very large number of pendulum swings
#so that the uncertainty in the period is negligible


pi2dTsqrd = ((2*np.pi)/np.array(T))**2

<a name='problems_2_3'></a>   

| [Top](#section_2_0) | [Restart Section](#section_2_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem 2.3.1</span>

Using an approach similar to the Johnson noise example, calculate the error on Newton's gravitational constant in this experiment as a percent. Enter your answer as a percentage with precision 1e-3 (for instance an answer of 7.25% would be entered as 7.250). Use the code below as a starting point.


In [ ]:
#>>>PROBLEM: P2.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

np.random.seed(2)
from lmfit.models import Model

def compute(m_earth, m_earth_unc, r_earth, r_earth_unc, l, l_unc, n_samp):
    samples = []
    ########################
    ### INSERT CODE HERE ###
    ########################
    return np.array(samples)


def get_rgr_data(m_earth, m_earth_unc, r_earth, r_earth_unc, l, l_unc, n_samp):
    rgr = []
    rgr_unc = []
    for k in range(10):
        ########################
        ### INSERT CODE HERE ###
        ########################
        
    rgr = np.array(rgr)  
    rgr_unc = np.array(rgr_unc)
    return rgr, rgr_unc


def linear_model(x, G, b):
    #the fit function again includes a non-zero intercept
    return ######INSERT CODE HERE########


n_samp = 100
rgr, rgr_unc = get_rgr_data(m_earth, m_earth_unc, r_earth, r_earth_unc, l, l_unc, n_samp)
lmod = Model(linear_model)
lmod.set_param_hint(name = 'G', value = 1)
lmod.set_param_hint(name = 'b', value = 0.1)
result = lmod.fit(1e-11*rgr, x = pi2dTsqrd, weights = 1/(rgr_unc*1e-11))
print(result.fit_report())
result.plot_fit()
plt.show()

>#### Follow-up 2.3.1a (ungraded)
>
>Compare your calculation of $G$ using the numbers given above to the current accepted value of Newton's gravitational constant. Where did the majority of your error come from?